In [1]:
# libraries
import os
import requests
import threading
import pandas as pd
from bs4 import BeautifulSoup as bs4
from requests.adapters import HTTPAdapter

# from Project
from GetSheetStatement import Constractv3


class BalanceSheet(object):

    def __init__(self, sheet: str, sid: str, year: int, season: int):
        #super().__init__()
        self.sheet = Constractv2.FINANCIAL_STATMENT[sheet]
        self.url = Constractv2.MOPS_URL.format(sheet=self.sheet)
        Constractv2.HEADERS['Referer'].format(sheet=self.sheet)
        self.header = Constractv2.HEADERS
        self.payload = Constractv2.PAYLOAD
        self.payload ["co_id"] = sid
        self.payload["year"] = int(year)
        self.payload["season"] = f"0{season}"
        self.file_path = Constractv2.FILE_PATH.format(
            sheet=sheet, sid=self.payload["co_id"],year=self.payload["year"] + 1911, season=self.payload["season"])
        if not os.path.exists(sheet):
            os.mkdir(sheet)

        print(f"INIT Thread: Balance_Sheet\nURL: {self.url}\nFILE_PATH: {self.file_path}\n")
        pass

    def run(self):
        # set session retry option
        web_ss = requests.session()
        ss_adapter = HTTPAdapter(max_retries=3)
        web_ss.mount("https://", adapter=ss_adapter)
        try:
            # start get html
          
            res = web_ss.post(url=self.url, headers=self.header, data=self.payload, timeout=3)
            
            # check iff: Response OK
            if res.status_code == 200:
                soup = bs4(res.text, "html.parser")
                tables = soup.select('table.hasBorder')
                #print(tables)
                return tables
            
        except Exception as e:
            print(f"Get HTML FAIL: {e}", end="\n\n")


#B is 資產負債表
#sheet = 'B'

#P is 損益表
#sheet= 'P' 

#C is 現金流量表


sheet= 'C' 
sid = '1101'
year = 108
season= 4

BSS=BalanceSheet(sheet, sid, year, season)
result=BSS.run()

even = result[0].select('td.even')
odd = result[0].select('td.odd')

#for i, v in enumerate(even):
#    print(i, v.text)


# %k
# B: 1, 2, 3 = 5
# B: 4 = 7
# P: 1, 2, 3, 4 = 5
# C = 3

if sheet=='C':
    k = 3
else:
    if season==4 or sheet == 'P':
        k=5
    else:
        k=7

INIT Thread: Balance_Sheet
URL: https://mops.twse.com.tw/mops/web/ajax_t164sb05
FILE_PATH: ./C/1101-2019-04.json



In [2]:
columns_odd=[]

for i, v in enumerate(even):
    if i% k ==0 :
        columns_odd.append(v.text.strip())
#print(columns_odd , len(columns_odd))

columns_even=[]
for i, v in enumerate(odd):
    if i% k ==0 :
        columns_even.append(v.text.strip())
        
#print(columns_even , len(columns_even))

columns=[]
for _o, _e in zip(columns_odd, columns_even):
    columns.append(_o)
    columns.append(_e)
    
print(columns, len(columns))

['營業活動之現金流量－間接法', '繼續營業單位稅前淨利（淨損）', '折舊費用', '本期稅前淨利（淨損）', '利息費用', '攤銷費用', '股利收入', '透過損益按公允價值衡量金融資產及負債之淨損失（利益）', '處分及報廢不動產、廠房及設備損失（利益）', '利息收入', '處分投資性不動產損失（利益）', '股份基礎給付酬勞成本', '處分投資損失（利益）', '採用權益法認列之關聯企業及合資損失（利益）之份額', '非金融資產減損損失', '非金融資產減損迴轉利益', '未實現外幣兌換損失（利益）', '收益費損項目合計', '其他項目', '應收票據（增加）減少', '強制透過損益按公允價值衡量之金融資產（增加）減少', '應收帳款（增加）減少', '合約資產（增加）減少', '其他應收款（增加）減少', '應收帳款－關係人（增加）減少', '存貨（增加）減少', '預付款項（增加）減少', '其他流動資產（增加）減少', '其他流動負債增加（減少）', '與營業活動相關之資產之淨變動合計', '與營業活動相關之負債之淨變動合計', '應付帳款增加（減少）', '調整項目合計', '其他應付款增加（減少）', '退還（支付）之所得稅', '淨確定福利負債增加(減少)', '投資活動之現金流量', '與營業活動相關之資產及負債之淨變動合計', '處分透過其他綜合損益按公允價值衡量之金融資產', '營運產生之現金流入（流出）', '取得按攤銷後成本衡量之金融資產', '營業活動之淨現金流入（流出）', '取得採用權益法之投資', '取得透過其他綜合損益按公允價值衡量之金融資產', '採用權益法之被投資公司減資退回股款', '處分採用權益法之投資', '處分不動產、廠房及設備', '取得不動產、廠房及設備', '取得使用權資產', '取得無形資產', '長期應收租賃款減少', '其他非流動資產增加', '收取之股利', '其他預付款項增加', '其他投資活動', '收取之利息', '籌資活動之現金流量', '投資活動之淨現金流入（流出）', '短期借款減少', '應付短期票券增加', '發行公司債', '償還長期借款', '舉借長期借款', '租賃本金償還', '發放現金股利', '其他非流動負債減少', '庫藏股票買回成本', '現金增資', '處分

In [3]:
content_odd=[]
for i, v in enumerate(even):
    if i% k ==0 :
        content_odd.append(even[i+1].text.strip().replace(',',''))   
#print(content_odd)

content_even=[]
for i, v in enumerate(odd):
    if i% k ==0 :
        content_even.append(odd[i+1].text.strip().replace(',',''))   
#print(content_even)

content=[]
for _o, _e in zip(content_odd, content_even):
    content.append(_o)
    content.append(_e)
    
#print(content, len(content))

Newcolumns=['sID','Quarter']
Newcontent=[sid, season]
Newcolumns.extend(columns)
Newcontent.extend(content)

df=pd.DataFrame(columns=Newcolumns)

df.loc[1]=Newcontent


#json = df.to_json(orient='records')

df.to_json(BSS.file_path, orient='records')

df

,sID,Quarter,營業活動之現金流量－間接法,繼續營業單位稅前淨利（淨損）,折舊費用,本期稅前淨利（淨損）,利息費用,攤銷費用,股利收入,透過損益按公允價值衡量金融資產及負債之淨損失（利益）,...,處分子公司股權（未喪失控制力）,員工執行認股權,其他籌資活動,取得子公司股權,匯率變動對現金及約當現金之影響,支付之利息,期初現金及約當現金餘額,非控制權益變動,資產負債表帳列之現金及約當現金,籌資活動之淨現金流入（流出）
1,1101,4,,32633419,6909487,32633419,2199118,430278,-1448451,-179256,...,0,2121,-10675914,-1017,-1040887,-2102028,48507889,7621,53572620,-11641346
